In [1]:
!pip install git+https://github.com/jbloomAus/SAELens.git

  Cloning https://github.com/jbloomAus/SAELens.git to /tmp/pip-req-build-6ffhjkg2
  Running command git clone --filter=blob:none --quiet https://github.com/jbloomAus/SAELens.git /tmp/pip-req-build-6ffhjkg2
  Resolved https://github.com/jbloomAus/SAELens.git to commit 022f1deeec252d63990375432efde738cc92f3b1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install HFApi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.1 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install plotly nbformat ipykernel matplotlib seaborn numpy torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 9.0 MB/s eta 0:00:00


In [1]:
from transformer_lens.utils import tokenize_and_concatenate
from datasets import load_dataset  
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from sae_lens import SAE, ActivationsStore, HookedSAETransformer
from sae_lens.config import LanguageModelSAERunnerConfig
from sae_lens.evals import run_evals, EvalConfig
import pandas as pd
from tqdm import tqdm
import os

def get_available_gpu():
    num_gpus = torch.cuda.device_count()
    available_memory = []

    for i in range(num_gpus):
        # Get the total and free memory on this GPU
        total_memory = torch.cuda.get_device_properties(i).total_memory
        allocated_memory = torch.cuda.memory_allocated(i)
        free_memory = total_memory - allocated_memory
        available_memory.append(free_memory)
    
    # Select the GPU with the most available memory
    selected_gpu = available_memory.index(max(available_memory))
    return selected_gpu

# Set device to CUDA and clear cache
device = torch.device(f"cuda:{get_available_gpu()}" if torch.cuda.is_available() else "cpu")
if device == "cuda":
    torch.cuda.empty_cache()

# Load the base Gemma model
base_model_name = "google/gemma-2-2b"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, use_auth_token=True, torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained(base_model_name, use_auth_token=True)



# Load the pretrained SAE models
sae_release = "gemma-scope-2b-pt-res-canonical" 
sae_16k_id = "layer_22/width_16k/canonical" 
sae_65k_id = "layer_22/width_65k/canonical" 
sae_16k_tuple = SAE.from_pretrained(sae_release, sae_16k_id)
sae_65k_tuple = SAE.from_pretrained(sae_release, sae_65k_id)



# Extract SAE objects and move to CUDA
sae_16k = sae_16k_tuple[0].to(device)
sae_65k = sae_65k_tuple[0].to(device)

# # Configuration
# cfg = LanguageModelSAERunnerConfig(
#     model_name = base_model_name,
#     hook_name = "resid_post",  # TransformerLens hook for residual stream
#     hook_layer = 22,
#     dataset_path = "NeelNanda/pile-small-tokenized-2b",
#     is_dataset_tokenized = True,
#     context_size = 1024,
#     streaming = True,
#     d_in = base_model.config.hidden_size,
#     device = device,
#     dtype = "float16",
# )


/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [24]:
!pip install nnsight

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 12.1 MB/s eta 0:00:00


In [41]:
from nnsight import LanguageModel

import pickle
from scipy.cluster import hierarchy

sae_list = [sae_16k, sae_65k]
device = torch.device('cuda:0')

def calculate_linkages(sae, label):
    layer_index = 22
    print("decoder matrix dim", sae.W_dec.shape)
    decoders = sae.W_dec.view(24, -1, 3072).cpu().detach().numpy()
    
    linkages = {}
    roots = {}
    for setting in ['average', 'complete', 'weighted']:
        linkage_list = []
        root_list = []
        for layer in range(24):
            linkage = hierarchy.linkage(decoders[layer], method = setting, metric = 'cosine')
            root_list.append(hierarchy.to_tree(linkage))
            linkage_list.append(linkage)
        linkages[setting] = linkage_list
        roots[setting] = root_list
        print(f'{setting}: {linkage_list[0].shape} for each of {len(linkage_list)} layers')
    
    with open(f'linkages_{label}.pkl', 'wb') as f:
        pickle.dump(linkages, f)

    print("linkage calculation complete")


In [ ]:
calculate_linkages(sae_16k, "16k")
calculate_linkages(sae_65k, "65k")

decoder matrix dim torch.Size([16384, 2304])
average: (511, 4) for each of 24 layers
complete: (511, 4) for each of 24 layers
weighted: (511, 4) for each of 24 layers
linkage calculation complete
decoder matrix dim torch.Size([65536, 2304])
average: (2047, 4) for each of 24 layers
complete: (2047, 4) for each of 24 layers


In [34]:
# testing the integrity of pickle files


def test_pickle_file(filename):
    try:
        # Load the data from the pickle file
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        # Print some information about the loaded data
        print(f"Successfully loaded data from {filename}")
        print(f"Keys in the dictionary: {list(data.keys())}")

        print(len(data['average']))
        print(len(data['average'][0]))

        # Example inspection: check the shape of the first linkage matrix
        for setting in data:
            print(f"Inspecting linkage matrices for setting: {setting}")
            first_linkage = data[setting][0]
            print(f"Shape of the first linkage matrix for setting '{setting}': {first_linkage.shape}")
            break  # Stop after the first setting to keep it concise

    except Exception as e:
        print(f"Failed to load data from {filename}: {e}")

# Test the linkage pickle files
test_pickle_file('linkages_16k.pkl')
test_pickle_file('linkages_65k.pkl')

Successfully loaded data from linkages_16k.pkl
Keys in the dictionary: ['average', 'complete', 'weighted']
8
511
Inspecting linkage matrices for setting: average
Shape of the first linkage matrix for setting 'average': (511, 4)
Successfully loaded data from linkages_65k.pkl
Keys in the dictionary: ['average', 'complete', 'weighted']
8
2047
Inspecting linkage matrices for setting: average
Shape of the first linkage matrix for setting 'average': (2047, 4)


In [7]:
from nnsight import LanguageModel

torch.cuda.empty_cache()
# use "second" gpu (apparently 'cuda:1' gets used up first)

# Load the pretrained SAE models
sae_release = "gemma-scope-2b-pt-res-canonical" 
sae_16k_id = "layer_22/width_16k/canonical" 
sae_65k_id = "layer_22/width_65k/canonical" 
sae_16k_tuple = SAE.from_pretrained(sae_release, sae_16k_id)
sae_65k_tuple = SAE.from_pretrained(sae_release, sae_65k_id)

# Extract SAE objects and move to CUDA
sae_16k = sae_16k_tuple[0].to(device)
sae_65k = sae_65k_tuple[0].to(device)

sae_16k.eval()
sae_65k.eval()
print("type sae 16", type(sae_16k))
print("type sae 65", type(sae_65k))

# Set up activation store
activations_store = ActivationsStore.from_config(base_model, cfg)

def get_activations(tokenized_text):
    model = LanguageModel("google/gemma-2-2b", device_map=device)
    print(model)
    
    with model.trace(tokenized_text) as tracer:
        # Access the output of the 22nd transformer block
        activations = model.model.layers[22].output.save()
    
    return activations.value

dataset = load_dataset("NeelNanda/pile-small-tokenized-2b", streaming=True)
subset_of_pile = dataset['train'].take(100)


for sample in tqdm(subset_of_pile):
    #convert into tensor
    tensor_tokens = torch.tensor(sample['tokens'], dtype=torch.long).unsqueeze(0)
    print(type(tensor_tokens))
    
    output_activations = get_activations(tensor_tokens)
    activations_store.extend(output_activations)

# Assuming sae_16k and sae_65k are already defined and loaded
# Process with 16k SAE
encoded_activations_16k = sae_16k.encode_jumprelu(activations_store.activations)
decoded_activations_16k = sae_16k.decode_jumprelu(encoded_activations_16k)

# Process with 65k SAE
encoded_activations_65k = sae_65k.encode_jumprelu(activations_store.activations)
decoded_activations_65k = sae_65k.decode_jumprelu(encoded_activations_65k)

type sae 16 <class 'sae_lens.sae.SAE'>
type sae 65 <class 'sae_lens.sae.SAE'>


Resolving data files:   0%|          | 0/89 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/89 [00:00<?, ?it/s]

0it [00:00, ?it/s]

<class 'torch.Tensor'>


You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

0it [00:05, ?it/s]


AttributeError: 'ActivationsStore' object has no attribute 'extend'

In [6]:
# now let's calculate cosine sim

ImportError: cannot import name 'set_transformers_cache' from 'transformers' (/opt/conda/lib/python3.10/site-packages/transformers/__init__.py)

In [11]:
import os

# Calculate cosine similarity
cosine_similarity_16k = torch.nn.functional.cosine_similarity(encoded_activations_16k, re_encoded_activations_16k)
cosine_similarity_65k = torch.nn.functional.cosine_similarity(encoded_activations_65k, re_encoded_activations_65k)
print(f"Cosine similarity (16k): {cosine_similarity_16k.mean().item()}")
print(f"Cosine similarity (65k): {cosine_similarity_65k.mean().item()}")

# Add cosine similarities to metrics
metrics_16k['cosine_similarity'] = cosine_similarity_16k.mean().item()
metrics_65k['cosine_similarity'] = cosine_similarity_65k.mean().item()

# Save metrics
output_file = "gemma_2b_layer22_metrics.csv"
pd.DataFrame({'16k': pd.Series(metrics_16k), '65k': pd.Series(metrics_65k)}).to_csv(output_file)

# Print confirmation and file location
current_dir = os.getcwd()
print(f"Metrics saved to: {os.path.join(current_dir, output_file)}")
print(f"Current working directory: {current_dir}")

NameError: name 'encoded_activations_16k' is not defined